In [1]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-io

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import re

In [13]:
data = pd.read_csv('/kaggle/input/news-summary/news_summary_more.csv')

In [14]:
data.head()

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [5]:
data.columns

Index(['id', 'article', 'highlights'], dtype='object')

In [15]:
df = data
df = df.text.dropna()
df_articles = df.values.tolist()

# Limit the length of df_articles to 500
df_articles = df_articles[:500]

len(df_articles)


500

In [16]:
!pip install -U transformers
!pip install -U torch

  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_curand_cu11-10.2.10.91-py3-none-manylinux1_x86_64.whl (54.6 MB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (173.2 MB)
  Using cached triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Attempting uninstall: torch
    Found exi

# Summarization using T5 model

In [17]:
import torch
import os
import json

from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [18]:
T5_PATH = 'Einmalumdiewelt/T5-Base_GNAD'
t5_model = T5ForConditionalGeneration.from_pretrained(T5_PATH, output_past=True)
t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)
x = t5_tokenizer(df_articles, truncation=True, max_length=500)


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def t5_summarize(input_text, num_beams=4, num_words=80):
    input_text = ' '.join(input_text.split())
    input_tokenized = t5_tokenizer.encode(input_text, return_tensors="pt").to(device)
    summary_task = torch.tensor([[21603, 10]]).to(device)
    input_tokenized = torch.cat([summary_task, input_tokenized], dim=-1).to(device)

    # Move the model to the same device as input tensors
    t5_model.to(device)

    summary_ids = t5_model.generate(input_tokenized,
                                    num_beams=int(num_beams),
                                    no_repeat_ngram_size=3,
                                    length_penalty=2.0,
                                    min_length=30,
                                    max_length=int(num_words),
                                    early_stopping=True)

    output = [t5_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) 
              for g in summary_ids]
    return output[0]


In [20]:
%%time
for i in range(6):
    
        print('news articles ',i + 1, " : \n" )
        print(t5_summarize(df_articles[i]))
        print('............................................................................\n\n\n\n')


news articles  1  : 

Tech Mahindra's Saurav Kant, a graduate of upGrad's Online Power Learning program, is now a Data Scientist with a 90% salary hike.
............................................................................




news articles  2  : 

A Delhi techie has won a chance to win free food from Swiggy for a year. Users who spend more than 2000 CRED coins on a credit card will get a free meal.
............................................................................




news articles  3  : 

In the fourth ODI of the series, New Zealand defeated India by 8 wickets to win their first match of the five-match series.
............................................................................




news articles  4  : 

Aegon Life iTerm Insurance plan provides life cover up to the age of 100 years. Customers can also save up to â146,800 on taxes.
............................................................................




news articles  5  : 

"What if it's not true, the 

In [22]:
def summarize_text(text):
    # Generate summary using the T5 model
    summary = t5_summarize(text)
    
    # Show the generated summary
    print("\nSummary:")
    print(summary)

# Prompt the user for input and generate summary
user_input = input("Enter the text to summarize: ")
summarize_text(user_input)


Enter the text to summarize:  Millions of acres scorched by wildfires in the Western US and Canada — an area roughly the size of South Carolina — can be traced back to carbon pollution from the world’s largest fossil fuel and cement companies, scientists reported Tuesday.  The study by the Union of Concerned Scientists, published in the journal Environmental Research Letters, found that 37% of the area burned by wildfires in the West since 1986 — nearly 19.8 million acres out of 53 million — can be blamed on the planet-cooking pollution from 88 of the world’s major fossil fuel producers and cement manufacturers, the latter of which have been shown to produce around 7% of all carbon dioxide emissions.  The amalgam of megadrought and record-breaking heat that’s drying out vegetation due to climate change has stoked the West’s wildfires. And researchers found that since 1901, the fossil fuel activities of these companies, including ExxonMobil and BP, among others, warmed the planet by 0.5


Summary:
88 of the world’s major fossil fuel and cement manufacturers have been shown to produce around 7% of all carbon dioxide emissions, according to a new study. The pollution has stoked the West’s wildfires by heating the planet by nearly half of the global increase during that period.


# Summarization using bart model

In [23]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

In [24]:
bart_PATH = 'facebook/bart-large-cnn'
bart_model = BartForConditionalGeneration.from_pretrained(bart_PATH, output_past=True)
bart_tokenizer = BartTokenizer.from_pretrained(bart_PATH)
x = bart_tokenizer(df_articles, truncation=True, max_length=500)

In [25]:
def bart_summarize(input_text, num_beams=4, num_words=80):
    #input_text = str(input_text)
    input_text = ' '.join(input_text.split())
    input_tokenized = bart_tokenizer.encode(input_text, return_tensors='pt')
    summary_ids = bart_model.generate(input_tokenized,
                                      num_beams=int(num_beams),
                                      no_repeat_ngram_size=3,
                                      length_penalty=2.0,
                                      min_length=80,
                                      max_length=int(num_words),
                                      early_stopping=True)
    output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    return output[0]

In [26]:
%%time
for i in range(6):
    
        print('news article ',i + 1, " : \n" )
        print(bart_summarize(df_articles[i]))
        print('............................................................................\n\n\n\n')

news article  1  : 

Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. UpGrad's Online Power Learning has powered 3 lakh+
............................................................................




news article  2  : 

Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ix
............................................................................




news article  3  : 

New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday. India lost an international match under Rohit Sharma's captai

In [27]:
def summarize_text(text):
    # Generate summary using the bart model
    summary = bart_summarize(text)
    
    # Show the generated summary
    print("\nSummary:")
    print(summary)

# Prompt the user for input and generate summary
user_input = input("Enter the text to summarize: ")
summarize_text(user_input)

Enter the text to summarize:  Millions of acres scorched by wildfires in the Western US and Canada — an area roughly the size of South Carolina — can be traced back to carbon pollution from the world’s largest fossil fuel and cement companies, scientists reported Tuesday.  The study by the Union of Concerned Scientists, published in the journal Environmental Research Letters, found that 37% of the area burned by wildfires in the West since 1986 — nearly 19.8 million acres out of 53 million — can be blamed on the planet-cooking pollution from 88 of the world’s major fossil fuel producers and cement manufacturers, the latter of which have been shown to produce around 7% of all carbon dioxide emissions.  The amalgam of megadrought and record-breaking heat that’s drying out vegetation due to climate change has stoked the West’s wildfires. And researchers found that since 1901, the fossil fuel activities of these companies, including ExxonMobil and BP, among others, warmed the planet by 0.5


Summary:
Millions of acres scorched by wildfires in the Western US and Canada can be traced back to carbon pollution from the world’s largest fossil fuel and cement companies. The study by the Union of Concerned Scientists, published in the journal Environmental Research Letters, found that 37% of the area burned by wildfires since 1986 can be blamed on these companies. Since 1901, the fossil fuel


# **evaluation metric** #

In [28]:
!pip install rouge-score
!pip install rouge

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=dc7976ae232f98c5c36a64323845142562a5147e64ab55a824233603f894f369
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [29]:
from rouge import Rouge

# Define the generated summary and reference summary
generated_summary_T5 = '''88 of the world’s major fossil fuel and cement manufacturers have been shown to produce around 7% of all carbon dioxide emissions, according to a new study. The pollution has stoked the West’s wildfires by heating the planet by nearly half of the global increase during that period.'''
reference_summary = '''A study by the Union of Concerned Scientists reveals that carbon pollution from major fossil fuel and cement companies is responsible for approximately 37% of the wildfire-ravaged area in the Western US and Canada, equivalent to the size of South Carolina. The activities of these companies have contributed to a 0.5-degree Celsius increase in global temperature since 1901, exacerbating the megadrought and heat that fuel the wildfires in the region.'''

# Initialize Rouge scorer
rouge_scorer = Rouge()

# Calculate ROUGE scores
rouge_scores = rouge_scorer.get_scores(generated_summary_T5,
                                       reference_summary)


print("ROUGE Scores:")
print(rouge_scores)

ROUGE Scores:
[{'rouge-1': {'r': 0.35185185185185186, 'p': 0.4634146341463415, 'f': 0.3999999950936289}, 'rouge-2': {'r': 0.08695652173913043, 'p': 0.13043478260869565, 'f': 0.10434782128695674}, 'rouge-l': {'r': 0.24074074074074073, 'p': 0.3170731707317073, 'f': 0.27368420561994466}}]


In [30]:
from rouge import Rouge

# Define the generated summary and reference summary

generated_summary_bart = '''Millions of acres scorched by wildfires in the Western US and Canada can be traced back to carbon pollution from the world’s largest fossil fuel and cement companies. The study by the Union of Concerned Scientists, published in the journal Environmental Research Letters, found that 37% of the area burned by wildfires since 1986 can be blamed on these companies. Since 1901, the fossil fuel'''
reference_summary = '''A study by the Union of Concerned Scientists reveals that carbon pollution from major fossil fuel and cement companies is responsible for approximately 37% of the wildfire-ravaged area in the Western US and Canada, equivalent to the size of South Carolina. The activities of these companies have contributed to a 0.5-degree Celsius increase in global temperature since 1901, exacerbating the megadrought and heat that fuel the wildfires in the region.'''

# Initialize Rouge scorer
rouge_scorer = Rouge()

# Calculate ROUGE scores
rouge_scores = rouge_scorer.get_scores(generated_summary_bart,
                                       reference_summary)


print("ROUGE Scores:")
print(rouge_scores)

ROUGE Scores:
[{'rouge-1': {'r': 0.48148148148148145, 'p': 0.5416666666666666, 'f': 0.5098039165859286}, 'rouge-2': {'r': 0.2753623188405797, 'p': 0.31666666666666665, 'f': 0.29457363843519024}, 'rouge-l': {'r': 0.42592592592592593, 'p': 0.4791666666666667, 'f': 0.4509803871741639}}]
